In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .appName("Feture Enginering") \
        .getOrCreate()

## 🔢 1. `StringIndexer`

Converte valores categóricos (strings) em índices numéricos. Útil para algoritmos que só trabalham com números.

In [20]:
from pyspark.ml.feature import StringIndexer
dados = spark.createDataFrame([("SP",), ("RJ",), ("MG",), ("SP",),("ES",)], ["cidade"])
indexador = StringIndexer(inputCol="cidade", outputCol="cidade_indexado")
dados_indexados = indexador.fit(dados).transform(dados)
dados_indexados.show()

+------+---------------+
|cidade|cidade_indexado|
+------+---------------+
|    SP|            0.0|
|    RJ|            3.0|
|    MG|            2.0|
|    SP|            0.0|
|    ES|            1.0|
+------+---------------+



## 🔥 2. `OneHotEncoder`

Após indexar strings com `StringIndexer`, podemos transformar os índices em vetores binários. Isso evita dar peso ou ordem falsa aos valores.

In [25]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=["cidade_indexado"], outputCols=["cidade_oh"])
dados_oh = encoder.fit(dados_indexados).transform(dados_indexados)
dados_oh.show()

+------+---------------+-------------+
|cidade|cidade_indexado|    cidade_oh|
+------+---------------+-------------+
|    SP|            0.0|(3,[0],[1.0])|
|    RJ|            3.0|    (3,[],[])|
|    MG|            2.0|(3,[2],[1.0])|
|    SP|            0.0|(3,[0],[1.0])|
|    ES|            1.0|(3,[1],[1.0])|
+------+---------------+-------------+



In [29]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
dados = spark.createDataFrame([(Vectors.dense([10.0, 1000000.0]),),
                               (Vectors.dense([20.0, 2500000.0]),)], ["features"])
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True , withMean=True)
dados_escalados = scaler.fit(dados).transform(dados)
dados_escalados.show(truncate=False)

+----------------+-----------------------------------------+
|features        |scaled_features                          |
+----------------+-----------------------------------------+
|[10.0,1000000.0]|[-0.7071067811865475,-0.7071067811865475]|
|[20.0,2500000.0]|[0.7071067811865475,0.7071067811865475]  |
+----------------+-----------------------------------------+



In [37]:
dados = spark.createDataFrame([(25, 5000,120), (40, 12000,400)], ["idade", "salario", "horas_trabalhadas"])
dados.show()

+-----+-------+-----------------+
|idade|salario|horas_trabalhadas|
+-----+-------+-----------------+
|   25|   5000|              120|
|   40|  12000|              400|
+-----+-------+-----------------+



## 🧱 4. `VectorAssembler`

Combina múltiplas colunas em uma única coluna `features`. É obrigatório antes de treinar qualquer modelo em MLlib.

In [38]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["idade","salario"], outputCol="features")
dados_final = assembler.transform(dados)
dados_final.show()

+-----+-------+-----------------+--------------+
|idade|salario|horas_trabalhadas|      features|
+-----+-------+-----------------+--------------+
|   25|   5000|              120| [25.0,5000.0]|
|   40|  12000|              400|[40.0,12000.0]|
+-----+-------+-----------------+--------------+



In [36]:
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True , withMean=True)
dados_final = scaler.fit(dados_final).transform(dados_final)
dados_final.show(truncate=False)

+-----+-------+--------------+-----------------------------------------+
|idade|salario|features      |scaled_features                          |
+-----+-------+--------------+-----------------------------------------+
|25   |5000   |[25.0,5000.0] |[-0.7071067811865475,-0.7071067811865476]|
|40   |12000  |[40.0,12000.0]|[0.7071067811865475,0.7071067811865476]  |
+-----+-------+--------------+-----------------------------------------+



## 📉 7. `PCA` (Análise de Componentes Principais)

Reduz a dimensionalidade dos dados, mantendo a maior parte da variância. Útil para eliminar colunas redundantes ou muito correlacionadas.

In [41]:
dados = spark.createDataFrame([
    (Vectors.dense([1.0, 2.0, 3.0]),),
    (Vectors.dense([5.0, 6.0, 7.0]),),
    (Vectors.dense([9.0, 10.0, 11.0]),)
], ["features"])

In [55]:
from pyspark.ml.feature import PCA
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
modelo_pca = pca.fit(dados)
dados_pca = modelo_pca.transform(dados)
dados_pca.show(truncate=False)

+---------------+-----------------------------------------+
|features       |pcaFeatures                              |
+---------------+-----------------------------------------+
|[1.0,2.0,3.0]  |[-3.4641016151377544,-1.2247448713915887]|
|[5.0,6.0,7.0]  |[-10.392304845413262,-1.2247448713915894]|
|[9.0,10.0,11.0]|[-17.32050807568877,-1.2247448713915896] |
+---------------+-----------------------------------------+



In [56]:
print("Variância explicada por cada componente:", modelo_pca.explainedVariance.toArray())
print("Variância total explicada pelos", pca.getK(), "componentes:", sum(modelo_pca.explainedVariance))

Variância explicada por cada componente: [1.00000000e+00 8.52687525e-18]
Variância total explicada pelos 2 componentes: 1.0
